In [15]:
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec

ns_path = "test.namespace.yaml"
ext_source = "test.extensions.yaml"

ns_builder = NWBNamespaceBuilder('Test extension', "test", version='0.1.0')

ns_builder.include_type('NWBContainer', namespace='core')
ns_builder.include_type('Device', namespace='core')
ns_builder.include_type('NWBDataInterface', namespace='core')

slm = NWBGroupSpec(
        neurodata_type_def='SpatialLightModulator',
        neurodata_type_inc='NWBContainer',
        doc=('Spatial light modulator used in holographic photostimulation'),
        attributes=[
            NWBAttributeSpec('dimensions', 'dimensions ([w, h] or [w, h, d]) of SLM field', 'numeric', shape=((2, ), (3,)),
            )
        ])

ns_builder.add_spec(ext_source, slm)

from pynwb.spec import NWBDatasetSpec, NWBDtypeSpec, RefSpec

photostim_device = NWBGroupSpec(
        neurodata_type_def='PhotostimulationDevice',
        neurodata_type_inc='Device',
        doc=('photostimDevice'),
        attributes=[
            NWBAttributeSpec('type', 'type of stimulation (laser or LED)', 'text'),
            NWBAttributeSpec('wavelength', 'wavelength of photostimulation', 'numeric'),
            NWBAttributeSpec('slm', 'spatial light modulator device', RefSpec('GroupSpec', 'object'))
        ])

ns_builder.add_spec(ext_source, photostim_device)

photostim = NWBGroupSpec(
        neurodata_type_def='Photostimulation',
        neurodata_type_inc='NWBDataInterface',
        doc=('photostimulation container'),
        attributes=[
            NWBAttributeSpec('device', 'photostimulation device', RefSpec('GroupSpec', 'object')),
            NWBAttributeSpec('roi_coordinates', '[n,2] or [n,3] list of coordinates', 'numeric', shape=((None, 2), (None, 3))),
            NWBAttributeSpec('stimulation_diameter', 'diameter of stimulation (pixels)', 'numeric'),
            NWBAttributeSpec('roi_mask', 'mask of region of interest', 'numeric'),
            NWBAttributeSpec('opsin', 'opsin used', 'text'),
            NWBAttributeSpec('peak_pulse_power', 'peak pulse power (J)', 'numeric')
        ])

ns_builder.add_spec(ext_source, photostim)

ns_builder.export(ns_path)




In [24]:
from pynwb import register_class, load_namespaces
from pynwb.core import NWBContainer, NWBDataInterface
from collections.abc import Iterable
from pynwb.device import  Device
from hdmf.utils import docval, popargs, get_docval, get_data_shape, popargs_to_dict

ns_path = "test.namespace.yaml"
load_namespaces(ns_path)


@register_class('SpatialLightModulator', 'test')
class SpatialLightModulator(NWBContainer):

    __nwbfields__ = ('dimensions',)

    @docval(
        {'name': 'name', 'type': str, 'doc': 'name of spatial light modulator'},
        {'name': 'dimensions', 'type': Iterable, 'doc': 'dimensions ([w, h] or [w, h, d]) of SLM field'}
    )
    def __init__(self, **kwargs):
        dimensions = popargs('dimensions', kwargs)
        super().__init__(**kwargs)
        self.dimensions = dimensions

@register_class('PhotostimulationDevice', 'test')
class PhotostimulationDevice(Device):

    __nwbfields__ = ('type', 'wavelength', 'slm')

    @docval(*get_docval(Device.__init__) + (
        {'name': 'type', 'type': str, 'doc': 'type of stimulation (laser or LED)', 'default': None},
        {'name': 'wavelength', 'type': float, 'doc': 'wavelength of photostimulation', 'default': None},
        {'name': 'slm', 'type': SpatialLightModulator, 'doc': 'spatial light modulator', 'default': None},
        ))
    def __init__(self, **kwargs):
        keys_to_set = ("type",
                       "wavelength",
                       "slm")
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        for key, val in args_to_set.items():
            setattr(self, key, val)


@register_class('Photostimulation', 'test')
class Photostimulation(NWBDataInterface):

    # __nwbfields__ = ('device', 'roi_coordinates', 'stimulation_diameter')

    @docval(*get_docval(NWBDataInterface.__init__) + (
        {'name': 'device', 'type': PhotostimulationDevice, 'doc': 'photostimulation device'},
        {'name': 'roi_coordinates', 'type': Iterable, 'doc': '[n,2] or [n,3] list of coordinates', 'default': None},
        {'name': 'stimulation_diameter', 'type': (int, float), 'doc': 'diameter of stimulation (pixels)', 'default': None},
        {'name': 'roi_mask', 'type': 'array_data', 'doc': 'pixel mask for ROI', 'default': None},
        {'name': 'opsin', 'type': str, 'doc': 'opsin used', 'default': None},
        {'name': 'peak_pulse_power', 'type': (int, float), 'doc': 'peak pulse power (J)', 'default': None}
        ))
    def __init__(self, **kwargs):
        keys_to_set = ("device", "roi_coordinates",
                       "stimulation_diameter", "roi_mask", "opsin", "peak_pulse_power")
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        for key, val in args_to_set.items():
            setattr(self, key, val)

        if (self.roi_coordinates is None and self.stimulation_diameter is None) and (self.roi_mask is None):
            raise TypeError("roi_coordinates & stimulation_diameter OR roi_mask must be specified")




In [28]:

from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, \
    Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries

import matplotlib.pyplot as plt

import importlib
import ndx_photostim
importlib.reload(ndx_photostim)

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

import numpy as np
slm = SpatialLightModulator(
    name="OpticalChannel",
    dimensions = [1, 2, 3],#[32, 10],
)

dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED', wavelength=320., slm=slm)
print(dev)

print(nwbfile)

nwbfile.add_device(dev)

stim = Photostimulation(name="photostim", device=dev, roi_coordinates=[[1, 2], [3, 4]], stimulation_diameter=5)

nwbfile.add_acquisition(stim)

print(nwbfile)

device __main__.PhotostimulationDevice at 0x140526109246464
Fields:
  description: photostim_device
  slm: OpticalChannel __main__.SpatialLightModulator at 0x140524225511232
Fields:
  dimensions: [1 2 3]

  type: LED
  wavelength: 320.0

root pynwb.file.NWBFile at 0x140526379403776
Fields:
  file_create_date: [datetime.datetime(2022, 9, 28, 15, 11, 8, 242197, tzinfo=tzlocal())]
  identifier: EXAMPLE_ID
  session_description: my first synthetic recording
  session_start_time: 2022-09-28 15:11:08.241854-04:00
  timestamps_reference_time: 2022-09-28 15:11:08.241854-04:00

root pynwb.file.NWBFile at 0x140526379403776
Fields:
  acquisition: {
    photostim <class '__main__.Photostimulation'>
  }
  devices: {
    device <class '__main__.PhotostimulationDevice'>
  }
  file_create_date: [datetime.datetime(2022, 9, 28, 15, 11, 8, 242197, tzinfo=tzlocal())]
  identifier: EXAMPLE_ID
  session_description: my first synthetic recording
  session_start_time: 2022-09-28 15:11:08.241854-04:00
  timest

In [26]:
stim = Photostimulation(name="photostim", device=dev, peak_pulse_power=10, opsin='test')

TypeError: roi_coordinates & stimulation_diameter OR roi_mask must be specified

In [27]:
image_mask = np.zeros((100, 100))

# randomly generate example image masks
x = np.random.randint(0, 95)
y = np.random.randint(0, 95)
image_mask[x:x + 5, y:y + 5] = 1

stim = Photostimulation(name="photostim", device=dev, roi_mask=image_mask, peak_pulse_power=10, opsin='test')